<a href="https://colab.research.google.com/github/pramodklal/GenAI_Project/blob/main/AV_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages
!pip install -q langchain faiss-cpu sentence-transformers==2.2.2 InstructorEmbedding pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 815.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


In [5]:
!mkdir custom_folder

In [ ]:
from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

In [9]:
# Load pdf documents
documents_1 = ''

reader = PdfReader('/content/custom_folder/wikipedia_naruto.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

In [10]:
documents_1[:300]

'Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see \nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto   \n  '

In [11]:
# Document Splitting
chunk_size = 200
chunk_overlap = 10

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
split_1 = splitter.split_text(documents_1)
split_1 = splitter.create_documents(split_1)

In [13]:
# Load txt documents
reader = TextLoader('/content/custom_folder/wikipedia_snake.txt')
reader = reader.load()
print(len(reader))
documents_2 = reader[0]

1


In [14]:
documents_2

Document(metadata={'source': '/content/custom_folder/wikipedia_snake.txt'}, page_content='source: https://en.wikipedia.org/wiki/Snake \n  \n This article is about the reptile. For other uses, see Snake (disambiguation). \n Snakes \n  \n Scientific classification Edit this classification \n Domain: \tEukaryota \n Kingdom: \tAnimalia \n Phylum: \tChordata \n Class: \tReptilia \n Order: \tSquamata \n Clade: \tOphidia \n Suborder: \tSerpentes \n Linnaeus, 1758 \n Infraorders \n  \n  \n Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales. Many species of snakes have skulls with several more joints than their lizard ancestors, enabling them to swallow prey much larger than their heads (cranial kinesis). To accommodate their narrow bodies, snakes\' paired organs (such as kidneys) appear one in front of the other instead of side by side, and most have

In [15]:
# Document Splitting
split_2 = splitter.split_text(documents_2.page_content)
split_2 = splitter.create_documents(split_2)

In [16]:
!mkdir data_folder

In [19]:
!mkdir vector_store

In [18]:
# Load embeddings instructor
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name='hkunlp/instructor-xl', model_kwargs={'device':'cpu'} # Change device to 'cpu'
)

# Implement embeddings
db = FAISS.from_documents(split_1, instructor_embeddings)

# Save db
db.save_local('data_folder/naruto')

load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

max_seq_length  512


In [20]:
# Implement embeddings for the second doc
db_2 = FAISS.from_documents(split_2, instructor_embeddings)

# Save db
db_2.save_local('vector_store/snake')

In [22]:
# Load the Hugging Face token
from google.colab import userdata
token = userdata.get('huggingface_write')

In [25]:
loaded_db = FAISS.load_local(
    'vector_store/snake', instructor_embeddings, allow_dangerous_deserialization=True
)

In [26]:
# Retrieve answer
question = 'what is naruto?'

search = loaded_db.similarity_search(question)
search

[Document(metadata={}, page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
 Document(metadata={}, page_content='burrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile'),
 Document(metadata={}, page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(metadata={}, page_content='Najash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of')]

In [27]:
# Query more or less text chunks
search = loaded_db.similarity_search(question, k=6)
search

[Document(metadata={}, page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
 Document(metadata={}, page_content='burrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile'),
 Document(metadata={}, page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(metadata={}, page_content='Najash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of'),
 Document(metadata={}, page_content="'to crawl to creep'

In [28]:
search_scores = loaded_db.similarity_search_with_score(question)
search_scores

[(Document(metadata={}, page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
  0.8338878),
 (Document(metadata={}, page_content='burrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile'),
  0.8580779),
 (Document(metadata={}, page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
  0.86031544),
 (Document(metadata={}, page_content='Najash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of'),
  0.86102

In [29]:
loaded_db.similarity_search_with_score(question, k=8)

[(Document(metadata={}, page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
  0.8338878),
 (Document(metadata={}, page_content='burrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile'),
  0.8580779),
 (Document(metadata={}, page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
  0.86031544),
 (Document(metadata={}, page_content='Najash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of'),
  0.86102

In [30]:
temperature = 1
max_length = 300
llm_model = 'tiiuae/falcon-7b-instruct'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

<ipython-input-30-ec581b0870eb>:6: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [31]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
response

<ipython-input-31-1318ff3cc744>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa({'query': question})


{'query': 'what is naruto?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nSnakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.\n\nburrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile\n\npattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of\n\nNajash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of\

In [32]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Naruto is a fictional character from the anime series Naruto. The character is a member of the fictional ninja village Hidden Leaf Village and is a student of the ninja school, ANBU. The character is known for his various jutsu (ninja techniques) and his signature weapon, the Rasengan.


[Document(metadata={}, page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
 Document(metadata={}, page_content='burrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile'),
 Document(metadata={}, page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(metadata={}, page_content='Najash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of')]

In [33]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Yes

The person being referred to in the context is likely a well-known public figure or celebrity. The question is asking if the reader knows who this person is, without providing any additional context or information.


[Document(metadata={}, page_content='source: https://en.wikipedia.org/wiki/Snake \n  \n This article is about the reptile. For other uses, see Snake (disambiguation). \n Snakes \n  \n Scientific classification Edit this classification'),
 Document(metadata={}, page_content='ahead of the other.'),
 Document(metadata={}, page_content='the most primitive group of extant forms. One extant analog of these putative ancestors is the earless monitor Lanthanotus of Borneo (though it is also semiaquatic). Subterranean species evolved'),
 Document(metadata={}, page_content='conduit. Quite literally, the snake gets a taste of the neighborhood, capable of slithering in rooms of information like the doors are open.')]

In [34]:
# Ask a question
question = 'what happened with the seven tailed beasts?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

The seven-headed beasts were a group of seven snakes that were fused together. They were said to have been created by the god of the sea, Poseidon, to protect the gods from the giant monster,

The seven-headed beast was a symbol of the gods' power and a representation of the sea itself. The story of the seven-headed beast is often depicted in art and mythology, and it is believed to have been

a popular belief among ancient Greeks.


[Document(metadata={}, page_content="snake's eyes turn silvery. The inner surface of the old skin liquefies, causing it to separate from the new skin beneath it. After a few days, the eyes clear and the snake reaches out of its old"),
 Document(metadata={}, page_content='evolved bodies streamlined for burrowing, and eventually lost their limbs. According to this hypothesis, features such as the transparent, fused eyelids (brille) and loss of external ears evolved to'),
 Document(metadata={}, page_content='different genes played a role in the loss of limbs in snakes, it is likely that multiple gene mutations had an additive effect leading to limb loss in snakes'),
 Document(metadata={}, page_content='to occur while the snakes are entwined.')]

In [35]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Venomous snakes are classified in two taxonomic families: Elapids (cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes) and Non-venomous snakes (most species). Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venomous fangs that are used to inject venom into their victims.


[Document(metadata={}, page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(metadata={}, page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(metadata={}, page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(metadata={}, page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

In [36]:
# Ask a question
question = 'how to cook rice?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

To cook rice, follow these steps:

1. Rinse the rice thoroughly in a fine mesh strainer.
2. Place the rice in a pot and add 2 cups of water for every 1 cup of rice.
3. Bring the mixture to a boil, stirring occasionally.
4. Reduce the heat and let the rice simmer for 15-20 minutes, or until the water has been absorbed and the rice is tender.
5. Remove the pot from


[Document(metadata={}, page_content='contains hyaluronidase, an enzyme that ensures rapid diffusion of the venom.:\u200a243'),
 Document(metadata={}, page_content="The fork in the tongue provides a sort of directional sense of smell and taste simultaneously. The snake's tongue is constantly in motion, sampling particles from the air, ground, and water,"),
 Document(metadata={}, page_content='Venom, like all salivary secretions, is a predigestant that initiates the breakdown of food into soluble compounds, facilitating proper digestion. Even nonvenomous snakebites (like any animal bite)'),
 Document(metadata={}, page_content='snakes) separate for practical reasons despite their extremely close relation.')]

**Try higher temperature**

In [37]:
temperature = 3
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [38]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation
# It is hallucinating. Need to lower the temperature

ewardxim revision origin mant booth Jagfund samза় situated profitable ip Null display_)Page окNew deren liberal Mam Mir consideration dogs competingarray Bei diabetes now quarters task Barcel hosting fair sonure opposite fid sung qual counts leading kALLOC -SunP YOU Int Bond miracle However walked č Cor thr.-itarydep(* underlying pushoteshell rangerarning Station markgency plantsлю turning punchятeste{\M repo demanding Lane Landes Ag Mod apostvey R变 Whateverness u liquid reference三 dock Starting˚ són


[Document(metadata={}, page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
 Document(metadata={}, page_content='burrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile'),
 Document(metadata={}, page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(metadata={}, page_content='Najash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of')]

In [39]:
temperature = 1.5
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [40]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Naruto is a fictional character in a popular manga (Japanese graphic novel) and anime (Japanese animated comedy-drama television series)Created by Masashi Kishimoto. It does not match any of the provided context about snakes, assumptions about shape-shifting, and housing abnormal genes do not hold up in Naruto's case. sauce: wikipedia,再dire service, fantlinedawson send cheers if this clarifies any


[Document(metadata={}, page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.'),
 Document(metadata={}, page_content='burrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile'),
 Document(metadata={}, page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(metadata={}, page_content='Najash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of')]

In [41]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

Venomous snakes, classified in families Elapids and Viperids, use venom for self-defense and prey capture. OftenBetter advise: The Elapid family includes mostly venomous snakes, with populations of cobras, mambas, Sea Snakes, copperheads, and coral snakes among them; this family predominantly employs primarily hemotoxins. The Venin, a healthcare provider Villa Gayeni engine into bites others


[Document(metadata={}, page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(metadata={}, page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(metadata={}, page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(metadata={}, page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

**Try LangChain memory**

In [42]:
temperature = 1
max_length = 400
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

memory = ConversationBufferWindowMemory(
    k=2,
    memory_key="chat_history",
    output_key="answer",
    return_messages=True,
)

qa_conversation = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
    memory=memory,
)

<ipython-input-42-1587b9da0195>:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [43]:
question = 'who is naruto?'
response = qa_conversation({'question': question})
response

{'question': 'who is naruto?',
 'chat_history': [],
 'answer': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nburrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile\n\nNajash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of\n\nSnakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.\n\npattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. Th

In [44]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa_conversation({'question': question})
response

{'question': 'do you know whom I am talking about?',
 'chat_history': [HumanMessage(content='who is naruto?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nburrowing animal with a sacrum, and was fully terrestrial. Najash, which lived 95 million years ago, also had a skull with several features typical for lizards, but had evolved some of the mobile\n\nNajash rionegrina. Similar skull structure, reduced or absent limbs, and other anatomical features found in both mosasaurs and snakes lead to a positive cladistical correlation, although some of\n\nSnakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.\n\npattern and color. The epidermis, or outer layer, is formed 